# Narcos

The very mention of South America often evokes violent images from the Netflix series Narcos in which the story Pablo Escobar's rise to power and bloody empire in Medellin, Colombia is depicted. This project takes a look at the data, including violent crime, terror risk, attitudes towards women and natural disaster risl, to see if South America really is an exceptionally dangerous place to be.

## Import libraries

In [32]:
import pandas as pd

path = 'C:\\Users\\Jack-\\OneDrive\\Desktop\\Test\\'

## Import data

### Violent Crime

In [74]:
pd.read_csv(path + 'violent crime.csv')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 68


### Terror Risk

In [71]:
# reads in the data. This spreadsheet (political stability/lack of terrorism - World Bank) 
# would not read in properly so an average was taken from 2015-2022 manually using excel. 
TR_data = pd.read_csv(path + 'terror risk.csv')

# renames columns
TR_data.columns = ['country', 'terror_risk']

# shows the data
TR_data.head(6)

,country,terror_risk
0,Aruba,95.378483
1,Africa Eastern and Southern,NaN
2,Afghanistan,0.769317
3,Africa Western and Central,NaN
4,Angola,28.351302
5,Albania,54.805706


### Gender Violence

In [61]:
# reads in the data
raw_GV_data = pd.read_csv(path + 'sexual violence in the last 12 months (%).csv')

# filters for the most recent year
raw_GV_data = raw_GV_data[raw_GV_data['Year']==2018]

# removes uneeded columns
GV_data = raw_GV_data.iloc[:,[2,5]]

# renames columns
GV_data.columns = ['country', 'gender_violence']

# makes more desirable values higher
GV_data.loc[:,'gender_violence'] = 100 - GV_data.loc[:,'gender_violence']

# shows the data
GV_data.head(6)

,country,gender_violence
0,East Asia & Pacific,92.071015
1,Europe & Central Asia,93.682087
2,High income,94.398928
3,Latin America & Caribbean,91.693322
4,Low income,77.517801
5,Lower middle income,83.135147


### Natural Distaster

In [36]:
# reads in the data
raw_ND_data = pd.read_csv(path + 'worldriskindex-2023.csv')
raw_ND_data.head(6)

# removes uneeded columns
ND_data = raw_ND_data.iloc[:,[0,3]]

# renames columns
ND_data.columns = ['country', 'natural_distaster']

# makes more desirable values higher
ND_data.loc[:,'natural_distaster'] = 100 - ND_data.loc[:,'natural_distaster']

# shows the data
ND_data.head(6)

,country,natural_distaster
0,Afghanistan,95.98
1,Albania,93.77
2,Algeria,90.48
3,Andorra,99.78
4,Angola,88.96
5,Antigua and Barbuda,96.49
